In [47]:
##1. Setup Environment

In [48]:
!pip install tensorflow mediapipe opencv-python

In [49]:
##2. Data Preprocessing

In [50]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Load the dataset
file_path = 'squat.csv'
data = pd.read_csv(file_path)

# Separate features and labels
X = data.drop('class', axis=1)
y = data['class']

# Data augmentation function
def augment_data(X, y, angles=[15, 30, 45]):
    augmented_X = []
    augmented_y = []
    for angle in angles:
        rotation_matrix = np.array([[np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0],
                                    [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0],
                                    [0, 0, 1]])
        for i in range(X.shape[0]):
            pose = X.iloc[i].values.reshape(-1, 4)
            rotated_pose = np.dot(pose[:, :3], rotation_matrix.T)
            augmented_X.append(np.hstack((rotated_pose, pose[:, 3].reshape(-1, 1))).flatten())
            augmented_y.append(y.iloc[i])
    augmented_X = np.array(augmented_X)
    augmented_y = np.array(augmented_y)
    return np.vstack((X, augmented_X)), np.hstack((y, augmented_y))

# Augment the dataset
X_augmented, y_augmented = augment_data(X, y)

# Split the augmented dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [51]:
##3. Model Training

In [52]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(8, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50


C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2537 - loss: 2.2383 - val_accuracy: 0.7077 - val_loss: 1.1739
Epoch 2/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5585 - loss: 1.2946 - val_accuracy: 0.7808 - val_loss: 0.7897
Epoch 3/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6205 - loss: 1.0163 - val_accuracy: 0.8308 - val_loss: 0.6065
Epoch 4/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6919 - loss: 0.8244 - val_accuracy: 0.8231 - val_loss: 0.5322
Epoch 5/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7356 - loss: 0.6958 - val_accuracy: 0.8423 - val_loss: 0.4750
Epoch 6/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7163 - loss: 0.6880 - val_accuracy: 0.8346 - val_loss: 0.4362
Epoch 7/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7762 - loss: 0.6173 - val_accuracy: 0.8154 - val_loss: 0.4242
Epoch 8/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7719 - loss: 0.5639 - val_accuracy: 0.8500 - val_loss: 0.3937
Epo

In [53]:
##4. Real-Time Squat Evaluation

In [54]:
# Compute the mean pose for class 0
class_0_data = data[data['class'] == 0].drop('class', axis=1)
mean_pose_class_0 = class_0_data.mean().values

In [55]:
import cv2
import mediapipe as mp
import numpy as np
from sklearn.preprocessing import StandardScaler

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Function to extract landmarks
def extract_landmarks(landmarks):
    if landmarks is None:
        return np.zeros((33, 4)).flatten()
    return np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in landmarks.landmark]).flatten()

# Function to determine if the subject is performing a squat
def is_performing_squat(landmarks):
    if landmarks is None:
        return False
    visibility_threshold = 0.5
    # Check visibility of key landmarks (e.g., hips, knees, and ankles)
    key_points = [landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP],
                  landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP],
                  landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE],
                  landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE],
                  landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE],
                  landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]]
    return all([point.visibility > visibility_threshold for point in key_points])

# Precompute the mean pose for class 0
class_0_data = data[data['class'] == 0].drop('class', axis=1)
mean_pose_class_0 = class_0_data.mean().values

# Error descriptions mapping
error_descriptions = {
    0: "Correct squat movement",
    1: "Movement is too narrow",
    2: "Movement is too wide",
    3: "Movement is not low enough",
    4: "Movement is too low",
    5: "Knees past toes",
    6: "Heel raised",
    7: "Back is not straight"
}

# Real-time squat evaluation
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and find the pose
    results = pose.process(image)

    # Check if the subject is performing a squat
    if is_performing_squat(results.pose_landmarks):
        # Extract landmarks and make prediction
        landmarks = extract_landmarks(results.pose_landmarks)
        landmarks_normalized = scaler.transform([landmarks])
        prediction = model.predict(landmarks_normalized)
        class_id = np.argmax(prediction)

        # Calculate distance to mean pose of class 0
        distance_to_class_0 = np.linalg.norm(landmarks_normalized - mean_pose_class_0)
        max_distance = np.linalg.norm(np.max(X_train) - mean_pose_class_0)  # Max possible distance
        accuracy = max(0, 100 * (1 - distance_to_class_0 / max_distance))

        # Display feedback
        error_description = error_descriptions[class_id]
        feedback_text = f'Accuracy: {accuracy:.2f}%, Error: {error_description if class_id != 0 else "None"}'
        cv2.putText(frame, feedback_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    else:
        cv2.putText(frame, 'Please perform a squat', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Draw landmarks with green lines
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
        )

    # Show the image
    cv2.imshow('Squat Detection', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.G

...

C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [56]:
##5. Real-Time Evaluation

In [57]:
# import numpy as np

# # Function to extract landmarks
# def extract_landmarks(landmarks):
#     if landmarks is None:
#         return np.zeros((33, 4)).flatten()
#     return np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in landmarks.landmark]).flatten()

# # Real-time squat evaluation
# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Convert the image to RGB
#     image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#     # Process the image and find the pose
#     results = pose.process(image)

#     # Extract landmarks and make prediction
#     landmarks = extract_landmarks(results.pose_landmarks)
#     landmarks = scaler.transform([landmarks])
#     prediction = model.predict(landmarks)
#     class_id = np.argmax(prediction)
#     confidence = np.max(prediction)

#     # Display the result
#     cv2.putText(frame, f'Class: {class_id}, Confidence: {confidence:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
#     mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

#     # Show the image
#     cv2.imshow('Squat Detection', frame)

#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [58]:
import tensorflow as tf

# Save the model as a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to a file
with open('squat_model.tflite', 'wb') as f:
    f.write(tflite_model)

AttributeError: 'Sequential' object has no attribute '_get_save_spec'

In [59]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow as tf

# Define the model as before
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(8, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Define a function that will be used to convert the model to TensorFlow Lite
@tf.function(input_signature=[tf.TensorSpec(shape=[None, X_train.shape[1]], dtype=tf.float32)])
def model_predict(input_data):
    return {'outputs': model(input_data)}

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_concrete_functions([model_predict.get_concrete_function()])
tflite_model = converter.convert()

# Save the model to a file
with open('squat_model.tflite', 'wb') as f:
    f.write(tflite_model)


Epoch 1/50


C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
